In [1]:
from z3 import *
from meyer.meyer import conclude
from meyer.program import prog, progs
from meyer.feasibility import feasible
from meyer.equivalence import equal
from meyer.implementation import is_impl_of
from meyer.contracted_program import contracts
from meyer.conditionals import IfThenElse
from meyer.loop import fix_rep
from meyer.util.z3py_util import const, consts, U, UALL, elms
from meyer.util.z3py_set import set, sets
from meyer.util.z3py_rel import Empty as EmptyRel

[n1,n2,n3,n4,n5,n6,n7,n8] = UALL

s = Solver()

In [16]:
title = "adding 1 to a preconditon"
p1, p2, p3 = progs(s, "p1 p2 p3")

s.add(equal(p1, p2), p3.set() == p1.set(), p3.pre(1), p3.post() == EmptyRel())
conclude(s, p2 | p3 <= p1, title)

adding 1 to a preconditon
Universe = Int
Holds: unsat 



In [15]:
title = "addition contract"
p1, p2 = progs(s, "p1 p2")

x,y = Ints('x y')
s.add(p1.set() == p2.set(), p1.pre() == p2.pre())
s.add(+p1, ForAll([x,y], p1.post(x,y) == (y > x)))
s.add(+p2, ForAll([x,y], p2.post(x,y) == (y == x + 1)))

conclude(s, contracts(p2, p1), title)

Universe = Int
Holds: unsat 



In [3]:
title = "sum of 1st digit and 2nd digit" # multiple arguments!!
p1, p2 = progs(s, "p1 p2")
s.add(+p1, +p2)
s.add(p1.set() == p2.set(), p1.pre() == p2.pre())

x,y = Ints('x y')
s.add(ForAll([x, y], And(p1.post(x, y) == (x % 10 + x / 10 == y))))
s.add(ForAll([x, y], p2.post(x, y) == And([And(x == i, y == j) for i, j in [(24, 6)]])))

conclude(s, contracts(p2, p1), title)
# if there is at least one pair that matches p1's condition, p2 unfortunately becomes contracted program, otherwise unknown

sum of 1st digit and 2nd digit
Universe = Int
Holds: unsat 



In [7]:
title = "double" # p2 is an example program to double input value
p1, p2 = progs(s, "p1 p2")
x,y = Ints('x y')
s.add(+p1, +p2)
s.add(p1.set() == p2.set(), p1.pre() == p2.pre())

# s.add(ForAll(x, p1.set(x) == And(0 < x, x < 10)))
s.add(ForAll([x, y], p1.post(x, y) == (x * 2 == y)))
s.add(ForAll([x, y], p2.post(x, y) == Or([And(x == i, y == j) for i, j in [(1, 4)]])))
conclude(s, contracts(p2, p1), title)
# if there is at least one pair that matches p1's condition, p2 unfortunately becomes contracted program, otherwise unknown

double
Universe = Int
Unholds: unknown
smt tactic failed to show goal to be sat/unsat (incomplete quantifiers) 



In [4]:
title = "double"
p1, p2 = progs(s, "p1 p2")

# p1 and p2 are feasible
s.add(feasible(p1), feasible(p2))

# sets of p1 and p2 are equivalent
s.add(p1.set() == p2.set())

# preconditions of p1 and p2 are equivalent
s.add(p1.pre() == p2.pre())

# declare 2 elements x, y
x,y = elms('x y')

# double is a function that doubles input number
# n1 represents number 1, and other numbers as well
double = Function('double', U, U)
s.add([double(i)==j for i, j in 
    [(n1, n2), (n2, n4), (n3, n6), (n4, n8)]])

# p1 is a specification of double program.
s.add(ForAll([x, y], p1.post(x, y) == 
    (double(x) == y)))

# p2 is a concrete program of double program
s.add(ForAll([x, y], p2.post(x, y) == 
    Or([And(x == i, y == j) for i, j in [
        (n1, n2), (n2, n4)]])))

# check if p2 is a contracted program of p1
# this means p2 is a correct program on p1
conclude(s, contracts(p2, p1), title)

double
Universe = U, has 8 element(s)
Holds: unsat 



In [20]:
title = "If then else example"
p1, p2, p3 = progs(s, "p1 p2 p3")
# s.add(+p1, +p2, +p3)
# s.add(p1.set() == p2.set(), p2.set() == p3.set())
# s.add(p1.pre() == p2.pre(), p2.pre() == p3.pre())

c = set("c")
x, y = consts('x y', U)

ite1 = IfThenElse(c,p1,p2)
ite2 = IfThenElse(c,p3,p2)

is_positive = Function('is_positive', U, BoolSort())
s.add(ForAll(x, And([(is_positive(x) == (x == i)) for i in [n5, n6, n7, n8]])))
s.add(ForAll(x, c(x) == is_positive(x)))

s.add(ForAll([x, y], p1.post(x, y) == (y == x)))
s.add(ForAll([x, y], p2.post(x, y) == (y == n4)))
s.add(ForAll([x, y], p3.post(x, y) == (y == n5)))

conclude(s, contracts(ite2, ite1), title)

If then else example
Universe = U, has 8 element(s)
Holds: unsat 



In [2]:
title = "loop"
p1, p2 = progs(s, "p1 p2")
s.add(+p1, +p2)
s.add(p1.set() == p2.set())

x,y = consts('x y', U)
s.add(ForAll([x, y], p1.post(x, y) == Or([And(x == i, y == j) for i, j in [
    (n1, n2), (n2, n4), (n3, n6), (n4, n8)]])))
s.add(ForAll([x, y], p2.post(x, y) == Or([And(x == i, y == j) for i, j in [
    (n1, n2), (n2, n4)]])))
lp1 = fix_rep(p1, 2)
lp2 = fix_rep(p2, 2)
conclude(s, contracts(lp2, lp1), title)

loop
Universe = U, has 8 element(s)
Unholds: sat
set of p1_1
 else -> True

pre of p1_1
 else -> Or(If(Var(0) == U7,
      U7,
      If(Var(0) == U6,
         U6,
         If(Var(0) == U8,
            U8,
            If(Var(0) == U1,
               U1,
               If(Var(0) == U2,
                  U2,
                  If(Var(0) == U4,
                     U4,
                     If(Var(0) == U3, U3, U5))))))) ==
   U4,
   If(Var(0) == U7,
      U7,
      If(Var(0) == U6,
         U6,
         If(Var(0) == U8,
            U8,
            If(Var(0) == U1,
               U1,
               If(Var(0) == U2,
                  U2,
                  If(Var(0) == U4,
                     U4,
                     If(Var(0) == U3, U3, U5))))))) ==
   U2,
   If(Var(0) == U7,
      U7,
      If(Var(0) == U6,
         U6,
         If(Var(0) == U8,
            U8,
            If(Var(0) == U1,
               U1,
               If(Var(0) == U2,
                  U2,
                  If(Var(0) =